In [1]:
import json
import numpy as np
import pandas as pd
import os
import twitter

from twitter_auth import get_secret

In [14]:
import argparse

In [16]:
parser = argparse.ArgumentParser()

In [ ]:
parser.add_argument("--name", "-nm", name=)

In [2]:
f = "./woeids.json"

In [3]:
woeids = pd.read_json(f)

In [4]:
woeids.head()

,country,countryCode,name,parentid,placeType,url,woeid
0,,None,Worldwide,0,"{'code': 19, 'name': 'Supername'}",http://where.yahooapis.com/v1/place/1,1
1,Canada,CA,Winnipeg,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/2972,2972
2,Canada,CA,Ottawa,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/3369,3369
3,Canada,CA,Quebec,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/3444,3444
4,Canada,CA,Montreal,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/3534,3534


In [5]:
woeids.loc[woeids['name'] == 'Charlotte']

,country,countryCode,name,parentid,placeType,url,woeid
342,United States,US,Charlotte,23424977,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/2378426,2378426


In [6]:
def lookup(name, country='United States'):
    name = name.title()
    country = country.title()
    try:
        return woeids.loc[(woeids['country'] == country) & (woeids['name'] == name), 'woeid'].iloc[0]
    except IndexError: print("Not a valid country/name combo.")

In [7]:
lookup('milwaukee')

2451822

In [8]:
twit_auth = get_secret()

In [9]:
consumer_key = twit_auth['CONSUMER_KEY']
consumer_secret = twit_auth['CONSUMER_SECRET']
access_token = twit_auth['ACCESS_TOKEN']
access_secret = twit_auth['ACCESS_SECRET']

In [10]:
api = twitter.Api(consumer_key, consumer_secret, access_token, access_secret)

In [11]:
def trending_by_geo(name, country="United States"):
    woeid = lookup(name, country)
    try:
        raw = [(x.name, x.volume) for x in api.GetTrendsWoeid(woeid)]
    except:
        print("Not a Twitter geographical category.")
    else:
        filtered = []
        for result in raw:
            if result[1] != None:
                filtered.append(result)
        results = pd.DataFrame(data=filtered, columns=["HashTag", "TweetVol"])
        results = results.sort_values("TweetVol", ascending=False)
        results.index = range(1, len(results)+1)
        print("Top Trending HashTags in {}".format(name.title()))
        return results

In [12]:
trending_by_geo("milwaukee")

Top Trending HashTags in Milwaukee


,HashTag,TweetVol
1,Biden,891264
2,Democrats,278848
3,Minnie,130179
4,Rodman,127242
5,Lebron,75234
6,Lakers,67429
7,Ligue,58172
8,Bulls,38025
9,#AliensExist,34581
10,Adina,32900


In [13]:
api.GetStatus(1254250976015290369)

Status(ID=1254250976015290369, ScreenName=bookofthenewsun, Created=Sun Apr 26 03:28:30 +0000 2020, Text='@TalbertSwan @realDonaldTrump You’re a pastor?')